In [ ]:
%run "C:\Users\james\Documents\MLB\Code\U1. Imports.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U2. Utilities.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"


baseball_path = r'C:\Users\james\Documents\MLB\Database'

db_path = r'C:\Users\james\Documents\MLB\Database\MLBDB.db'
engine = create_engine(f'sqlite:///{db_path}')

In [ ]:
Maybe use base model? Just merge in sba, sbr onto runner_id

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LinearRegression

In [ ]:
import statsmodels.formula.api as smf
import pickle

In [ ]:
%run "A02. MLB API.ipynb"
%run "A03. Steamer.ipynb"

In [ ]:
complete_dataset = merge_datasets_sb(2023, 2023, 50, 300)

In [ ]:
# Read in Steamer hitters 
steamer_hitters_df = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_hitters_weekly_log.csv"), encoding='iso-8859-1')
# Clean
steamer_hitters_df2 = clean_steamer_hitters(steamer_hitters_df)
steamer_hitters_df2.dropna(subset=batter_stats_fg, inplace=True)

In [ ]:
# Find the dates of Steamer projections
# We'll take the most recent and merge in that projection for each player
batter_steamer_dates = list(steamer_hitters_df2['date'].unique())

# Define a function to find the largest number in "steamer_dates" less than or equal to a given "date"
def find_steamer_date(date, steamer_dates):
    max_steamer_date = max(filter(lambda d: d <= date, steamer_dates), default=None)
    return max_steamer_date

# Apply the function to create the "steamer_date" column in your DataFrame
complete_dataset["batter_date"] = complete_dataset["date"].apply(lambda x: find_steamer_date(x, batter_steamer_dates))

In [ ]:
# Steamer stats we want to keep
batter_stats_fg_plus = ['mlbamid', 'steamerid', 'date'] + batter_stats_fg + ['sba_imp', 'sbr']

# Merge
complete_merged_df = pd.merge(complete_dataset, steamer_hitters_df2[batter_stats_fg_plus], left_on=['batter', 'batter_date'], right_on=['mlbamid', 'date'], how='inner')

In [ ]:
complete_merged_df.tail(50)

In [ ]:
def create_steal_inputs(df):
    # Create on base variables
    df['preOnFirst'] = df.groupby(['gamePk', 'inning', 'halfInning'])['postOnFirst'].shift(1)
    df['preOnSecond'] = df.groupby(['gamePk', 'inning', 'halfInning'])['postOnSecond'].shift(1)
    df['preOnThird'] = df.groupby(['gamePk', 'inning', 'halfInning'])['postOnThird'].shift(1)

    df['onFirst'] = df['preOnFirst'].notnull().astype('int')
    df['onSecond'] = df['preOnSecond'].notnull().astype('int')
    df['onThird'] = df['preOnThird'].notnull().astype('int')
    
    # Create steal variables
    df['sb2b'] = np.where(df['movementReason'] == "r_stolen_base_2b", 1, 0)
    df['sb3b'] = np.where(df['movementReason'] == "r_stolen_base_3b", 1, 0)
    df['sbh'] = np.where(df['movementReason'] == "r_stolen_base_home", 1, 0)

    df['cs2b'] = np.where((df['movementReason'] == "r_caught_stealing_2b") | (df['movementReason'] == "r_pickoff_caught_stealing_2b"), 1, 0)
    df['cs3b'] = np.where((df['movementReason'] == "r_caught_stealing_3b") | (df['movementReason'] == "r_pickoff_caught_stealing_3b"), 1, 0)
    df['csh'] = np.where((df['movementReason'] == "r_caught_stealing_home") | (df['movementReason'] == "r_pickoff_caught_stealing_home"), 1, 0)
    
    # Create running season totals
    running = df.groupby('id')['sb2b', 'sb3b', 'sbh', 'cs2b', 'cs3b', 'csh'].sum().reset_index()
    running['sb'] = running[['sb2b', 'sb3b', 'sbh']].sum(axis=1)
    running['cs'] = running[['cs2b', 'cs3b', 'csh']].sum(axis=1)
    running['sbr'] = running['sb'] / (running['sb'] + running['cs'])
    
    # Create season opportunity totals
    opp_sb2b = df[(df['onFirst'] == 1) & (df['onSecond'] == 0)]
    opp_sb2b = opp_sb2b.groupby('preOnFirst')['onFirst'].sum().reset_index()

    opp_sb3b = df[(df['onSecond'] == 1) & (df['onFirst'] == 0)]
    opp_sb3b = opp_sb3b.groupby('preOnSecond')['onSecond'].sum().reset_index()
    
    running = running.merge(opp_sb2b, left_on='id', right_on='preOnFirst', how='left')
    running = running.merge(opp_sb3b, left_on='id', right_on='preOnSecond', how='left')
    
    running.drop(columns={'preOnFirst', 'preOnSecond'}, inplace=True)
    running.fillna(0, inplace=True)
    running['sba2b'] = running['sb2b'] + running['cs2b']
    running['sba3b'] = running['sb3b'] + running['cs3b']

    # Calculate attempt rate
    running['sba_2b'] = running['sba2b'] / running['onFirst']
    running['sba_3b'] = running['sba3b'] / running['onSecond']

    # If missing, assume they'll go at the 50th percentile
    running['sba_2b'].fillna(0.015873, inplace=True)
    running['sba_3b'].fillna(0.0, inplace=True)

    # Calculate success rate
    running['sb_2b'] = running['sb2b'] / running['sba2b']
    running['sb_3b'] = running['sb3b'] / running['sba3b']

    # Assume they make it 50% of the time (50th percentile)
    running['sb_2b'].fillna(0.5, inplace=True)
    running['sb_3b'].fillna(0.5, inplace=True)

    running.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Use at bat data to estimate times on first base
    # This will serve as a proxy for stolen base opportunities
    ab_df = df.drop_duplicates(['gamePk', 'atBatIndex'], keep='first')
    ab_df['reachedFirst'] = np.where(((ab_df['event'] == 'Walk') | (ab_df['event'] == "hit_by_pitch") | (ab_df['event'] == 'Single')), 1, 0)
    ab_df = ab_df.groupby('batter')['reachedFirst'].sum().reset_index()
    
    running = running.merge(ab_df, left_on='id', right_on='batter', how='left')
    
    running = running[['id', 'sb', 'cs', 'sbr', 'reachedFirst', 'sba_2b', 'sba_3b', 'sb_2b', 'sb_3b']]
    
    return running

In [ ]:
def create_sample(start_year, end_year):
    year_list = []
    for year in range(start_year, end_year):
        filename = "Play" + str(year) + ".csv"
        df = pd.read_csv(os.path.join(baseball_path, "API", "Raw", filename))
        running = create_steal_inputs(df)
        year_list.append(running)
        
    sample = pd.concat(year_list, axis=0)
    
    return sample  

# Models

In [ ]:
# Create sample
sample = create_sample(2013, 2022)
# Select players with large enough sample size
sample = sample[sample['reachedFirst'] > 50]
print(sample['sba_2b'].describe())
# Calculate implied stolen base attempt rate
sample['sba_imp'] = (sample['sb'] + sample['cs']) / sample['reachedFirst']

In [ ]:
# Attempt steal of 2B
X = sample[['sba_imp']]
y = sample['sba_2b']

sba_2b_reg = LinearRegression().fit(sample[['sba_imp']], sample['sba_2b'])

sample['sba_2b_hat'] = sba_2b_reg.predict(X)

filename = "sba_2b_20220901.sav"
pickle.dump(sba_2b_reg, open(filename, 'wb'))

sample[['sba_2b', 'sba_2b_hat']].describe()

In [ ]:
sample.sort_values(['sba_2b'], inplace=True)
sample.plot.scatter(x='sba_2b_hat', y='sba_2b')

In [ ]:
# Attempt steal of 3B
X = sample[['sba_imp']]
y = sample['sba_3b']

sba_3b_reg = LinearRegression().fit(sample[['sba_imp']], sample['sba_3b'])

sample['sba_3b_hat'] = sba_3b_reg.predict(X)

filename = "sba_3b_20220901.sav"
pickle.dump(sba_3b_reg, open(filename, 'wb'))

sample[['sba_3b', 'sba_3b_hat']].describe()

In [ ]:
sample.sort_values(['sba_3b'], inplace=True)
sample.plot.scatter(x='sba_3b_hat', y='sba_3b')

In [ ]:
# Thsi is very good for 2b. 3b is messy but idk if I care

In [ ]:
# Successful steal of 2B
X = sample[['sbr']]
y = sample['sb_2b']

sb_2b_reg = LinearRegression().fit(sample[['sbr']], sample['sb_2b'])

sample['sb_2b_hat'] = sb_2b_reg.predict(X)

filename = "sb_2b_20220901.sav"
pickle.dump(sb_2b_reg, open(filename, 'wb'))

sample[['sb_2b', 'sb_2b_hat']].describe()

In [ ]:
sample.sort_values(['sb_2b'], inplace=True)
sample.plot.scatter(x='sb_2b_hat', y='sb_2b')

In [ ]:
# Successful steal of 3B
X = sample[['sbr']]
y = sample['sb_3b']

sb_3b_reg = LinearRegression().fit(sample[['sbr']], sample['sb_3b'])

sample['sb_3b_hat'] = sb_3b_reg.predict(X)

filename = "sb_3b_20220901.sav"
pickle.dump(sb_3b_reg, open(filename, 'wb'))

sample[['sb_3b', 'sb_3b_hat']].describe()

In [ ]:
sample.sort_values(['sb_3b'], inplace=True)
sample.plot.scatter(x='sb_3b_hat', y='sb_3b')